In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
import csv
from datetime import datetime, time

# Data source: COVID19 Tracking


### CSV File available on GitHub

https://github.com/COVID19Tracking/covid-tracking-data/blob/master/data/states_daily_4pm_et.csv

### Page describing the project
https://covidtracking.com


In [2]:
#Upload the data from the COVID19Tracking project GitHub page

#Upload the state-by-state USA data
url_states = 'https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv'
covid19_usa_states_daily = pd.read_csv(url_states)
#Upload the USA data
url_usa = 'https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/us_daily.csv'
covid19_usa_daily = pd.read_csv(url_usa)

### Data available in the USA database

- **date** = date of observation
- **states** number of states with covid-19 cases
- **positive** number of tests with positive result
- **negative** number of tests with negative result
- **posNeg** sum of positive and negative
- **pending** number of tests with pending result
- **death** number of death cases
- **total** total number of cases

# Create a unique database with the state values and the USA values

In [3]:
#Change the column 'states', which previously tracked the number of infected states, into a column
#'state', all called USA. In this way USA will be read as one of the possible states for which one can 
# plot the available data, while in fact it will report the cumulative data, summed over all the states
covid19_usa_daily.columns = ['state' if x=='states' else x for x in covid19_usa_daily.columns]
covid19_usa_daily.state = 'USA'

#In this way the state-by-state and the federal DataFrame have the same columns. 

#Invert the direction of time for the federal database, which is ordered from the oldest to today, top to bottom.
covid19_usa_daily = covid19_usa_daily.iloc[::-1]
#In this way it is ordered as the state-wise database



#The states databases has some NaNs, for example in the number of deaths in states where it is zero. 
#So we fill it with a zero
covid19_usa_states_daily = covid19_usa_states_daily.fillna(0)
covid19_usa_daily = covid19_usa_daily.fillna(0)

#######STANDING ISSUE#############################
#total IS NOT EQUAL TO positive+negative+pending
#covid19_usa_daily['total']==(covid19_usa_daily['positive']+covid19_usa_daily['negative']+covid19_usa_daily['pending'])
#The difference is small so, for now, we proceed
#################################################

#Drop the dateChecked column from the state-wise database
covid19_usa_states_daily=covid19_usa_states_daily.drop(columns='dateChecked')

#Defin the posNeg column for the states database, to make it consistent with the USA database
covid19_usa_states_daily['posNeg']=covid19_usa_states_daily.positive+covid19_usa_states_daily.negative

#Reorder the columns in the states database, again for consistency with the USA database
covid19_usa_states_daily = covid19_usa_states_daily[['date','state','positive','negative','posNeg','pending','death','total']]


#Append the USA dataFrame to the state-wise dataFrame to create the actual dataFrame we will use
covid19_USA = covid19_usa_states_daily.append(covid19_usa_daily, ignore_index=True)

#Turn the string data into a datetime format, necessary to fit the data
covid19_USA['date']=pd.to_datetime(covid19_USA['date'],format="%Y%m%d")

In [4]:
#Seaborn setup for plots
sns.set(rc={'figure.figsize':(11, 7)})

# Data Analysis with the Verhulst growth model

In [5]:
#Define the function to use for curve_fit. Here we are using the Verhulst model
def Verhulst(t, K, I0, r):
    return (K*I0*np.exp(r*t))/(K+I0*(np.exp(r*t)-1))

In [6]:
# Function to create the numpy array of times associated to the data. The time-scale is in days and the 
#order of time in the data is (top ---> bottom) = (more recent ---> older)
def time_vector(data_frame,state_to_look):
    time_frame = data_frame[data_frame.state==state_to_look]['date'].to_numpy()
    t_vec = np.zeros(len(time_frame))
    for k in range(len(time_frame)):
        t_vec[k] = ((time_frame[0]-time_frame[k]).item())/(np.power(10,9)*60*60*24)
    return t_vec

In [7]:
# Function to fit the data, for a single state
def data_analysis_single(state_to_fit,indicator_to_fit):
    time_Data = time_vector(covid19_USA,state_to_fit)
    y_Data = np.flip(covid19_USA[covid19_USA.state==state_to_fit][indicator_to_fit].to_numpy())
    fittedParameters, covariance = curve_fit(Verhulst,time_Data,y_Data, bounds=([0,0,0],[300000000,1000,1]))
    return fittedParameters, covariance

In [8]:
#Full list of states and indicators to analyze

#Abbreviation of the states
state_list = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY","USA"]
indicators = list(covid19_USA.columns.values)[2:8]

## Fit with a Verhulst model and fill a csv with the value of the parameters and their error

### Fixed state

In [9]:
# Fixed State Fit
for k in state_list:
    with open('./Fit_Data/States/Verhulst_Parameters_{}.csv'.format(k), 'w', newline='') as f:
        thewriter = csv.writer(f)
        thewriter.writerow(['indicator','K','delta_K','r','delta_r','I0','delta_I0'])
        for n in indicators:
            try:
                p_opt, p_cov = data_analysis_single(k,n)
                p_err = np.sqrt(p_cov.diagonal())
                thewriter.writerow([n,p_opt[0],p_err[0],p_opt[2],p_err[2],p_opt[1],p_err[1]])
            except RuntimeError:
                thewriter.writerow([n,0,0,0,0,0,0])
                continue

### Fixed Indicator

In [10]:
# Fixed Indicator Fit
for n in indicators:
    with open('./Fit_Data/Indicators/Verhulst_Parameters_{}.csv'.format(n), 'w', newline='') as f:
        thewriter = csv.writer(f)
        thewriter.writerow(['state','K','delta_K','r','delta_r','I0','delta_I0'])
        for k in state_list:
            try:
                p_opt, p_cov = data_analysis_single(k,n)
                p_err = np.sqrt(p_cov.diagonal())
                thewriter.writerow([k,p_opt[0],p_err[0],p_opt[2],p_err[2],p_opt[1],p_err[1]])
            except RuntimeError:
                thewriter.writerow([n,0,0,0,0,0,0])
                continue